# 第14章 断言和测试

断言和测试是我们用来检查软件行为符合预期的两种重要手段。本章将向你展示用Scala编写和运行断言和测试的若干选择。

## 14.1 断言

在Scala中，断言的写法是对预定义方法assert的调用。[1]如 果condition不满足，表达式assert（condition）将抛出AssertionError。assert还有另一个版本：assert（condition, explanation），首先检查condition是否满足，如果不满足，那么就抛出包含给定explanation的 AssertionError。explanation的类型为Any，因此可以传入任何对象。assert方法将调用explanation的 toString方法来获取一个字符串的解释放入AssertionError。例如，在示例10.13（205页）的Element类中名为 “above”的方法，可以在对widen的调用之后加入一行断言来确保被加宽的（两个）元素具有相同的宽度。参考示例14.1。

In [2]:
import Element.elem
  
abstract class Element {

    def contents: Array[String]

    def width: Int =
      if (height == 0) 0 else contents(0).length

    def height: Int = contents.length

    def above(that: Element): Element = {
      val this1 = this widen that.width
      val that1 = that widen this.width
      assert(this1.width == that1.width)
      elem(this.contents ++ that.contents)        
    }

  
    def beside(that: Element): Element =
      elem(
        for (
          (line1, line2) <- this.contents zip that.contents
        ) yield line1 + line2
      )
    
  private def widen(w: Int): Element =
    if (w <= width) 
      this 
    else { 
      val left = elem(' ', (w - width) / 2, height) 
      var right = elem(' ', w - width - left.width, height) 
      left beside this beside right 
    } ensuring (w <= _.width)

  
    override def toString = contents mkString "\n"
  }

  object Element {
  
    def elem(contents: Array[String]): Element = 
      new ArrayElement(contents)
  
    def elem(chr: Char, width: Int, height: Int): Element = 
      new UniformElement(chr, width, height)
  
    def elem(line: String): Element = 
      new LineElement(line)
  }

  class ArrayElement(conts: Array[String]) extends Element {
    def contents: Array[String] = conts
  }

  class LineElement(s: String) extends ArrayElement(Array(s)) {
    override def width = s.length
    override def height = 1
  }

  class UniformElement(
    ch: Char, 
    override val width: Int,
    override val height: Int 
  ) extends Element {
    private val line = ch.toString * width
    def contents = Array.fill(height)(line)
  }


import Element.elem
  

defined class Element
defined object Element
defined class ArrayElement
defined class LineElement
defined class UniformElement

ensuring 这个方法可以被用于任何结果类型，这得益于一个隐式转换。虽然这段代码看上去调用的是widen结果的ensuring方法，实际上调用的是某个可以从 Element隐式转换得到的类型的ensuring方法。该方法接收一个参数，这是一个接收结果类型参数并返回Boolean的前提条件函数。 ensuring所做的，就是把计算结果传递给这个前提条件函数。如果前提条件函数返回true，那么ensuring就正常返回结果；如果前提条件返回 false，那么ensuring将抛出AssertionError。

在本例中，前提条件函数是“w <= _.width”。这里的下画线是传入该函数的入参的占位符，即调用widen方法的结果：一个Element。如果作为w传入widen方法的宽度小于 或等于结果Element的width，这个前提条件函数将得到true的结果，这样ensuring就会返回被调用的那个Element结果。由于这是 widen方法的最后一个表达式，widen本身的结果也就是这个Element了。

断言可以用JVM的命令行参数-e a和-d a来分别打开或关闭。打开时，断言就像是一个个小测试，用的是运行时得到的真实数据。在本章剩余的部分，我们将把精力集中在如何编写外部测试上，这些测试自己提供测试数据，并且独立于应用程序执行。

## 14.2 用Scala写测试

用Scala写测试，有很多选择，从已被广泛认可的Java工具，比如JUnit和TestNG，到用Scala编写的工具，比如ScalaTest、specs2和ScalaCheck。在本章剩余部分，我们将快速带你了解这些工具。我们从ScalaTest开始。

ScalaTest是最灵活的Scala测试框架：可以很容易地定制它来解决不同的问题。

ScalaTest的灵活性意味着团队可以使用任何最能满足他们需求的测试风格。例如，对于熟悉JUnit的团队，FunSuite风格是最舒适和熟悉的。参考示例14.3。